THIS CODE HAS BEEN WRITTEN ON COLAB PRO AND CHUNK PROCESSING HAS BEEN USED THROUGHOUT THE CODE TO AVOID ANY MEMORY ISSUES PLEASE ADJUST THE CHUNK SIZE ACCORDING TO YOUR SYSTEM MEMORY


In [ ]:
import pandas as pd
# Mount Google Drive to access datasets
from google.colab import drive
drive.mount('/content/drive')
# File paths for the datasets in Google Drive
communication_file = '/content/drive/My Drive/IIT_ROUND2/Train_r2/train_action_history.csv' # streaming the data from gdrive made it more accessible for my team please change the path accordingly from your gdrive
cdna_file = '/content/drive/My Drive/IIT_ROUND2/Train_r2/train_cdna_data.csv'
# Output path for the final merged dataset
output_file = '/content/drive/My Drive/merged_data.csv'
# Define chunk sizes
communication_chunksize = 500000  # these chunk sizes were selected in accordance with the available ram in colab pro please reduce it to approximately 25000 to use this code in the normal version
cdna_chunksize = 500000
# Prepare the output file (write header initially)
header_written = False
# Load and process Communication History in chunks
for comm_chunk in pd.read_csv(communication_file, chunksize=communication_chunksize):
    # Step 1: Decapitalize column names
    comm_chunk.columns = [col.lower() for col in comm_chunk.columns]

    # Step 2: Preprocess Communication History chunk
    comm_chunk['send_timestamp'] = pd.to_datetime(comm_chunk['send_timestamp']).dt.tz_localize(None)
    comm_chunk['open_timestamp'] = pd.to_datetime(comm_chunk['open_timestamp'])
    comm_chunk = comm_chunk.sort_values('send_timestamp')  # Sort for asof merge

    # Step 3: Load and process CDNA in chunks
    for cdna_chunk in pd.read_csv(cdna_file, chunksize=cdna_chunksize):
        # Step 3.1: Decapitalize column names
        cdna_chunk.columns = [col.lower() for col in cdna_chunk.columns]

        # Step 3.2: Preprocess CDNA chunk
        cdna_chunk['batch_date'] = pd.to_datetime(cdna_chunk['batch_date']).dt.tz_localize(None)
        cdna_chunk = cdna_chunk.sort_values('batch_date')

        # Step 4: Perform asof merge
        merged_chunk = pd.merge_asof(
            comm_chunk,
            cdna_chunk,
            left_on='send_timestamp',
            right_on='batch_date',
            by='customer_code',
            direction='backward'
        )

        # Step 5: Append merged chunk to the output file
        merged_chunk.to_csv(output_file, index=False, mode='a', header=not header_written)
        header_written = True  # Only write header for the first chunk


Mounted at /content/drive


<ipython-input-1-05b141c44fa0>:32: DtypeWarning: Columns (7,12,13,15,16,17,18,19,29,37,45,46,47,56,58,71,228) have mixed types. Specify dtype option on import or set low_memory=False.
  for cdna_chunk in pd.read_csv(cdna_file, chunksize=cdna_chunksize):
<ipython-input-1-05b141c44fa0>:32: DtypeWarning: Columns (7,12,13,15,16,17,18,19,37,45,46,47,56,58,66,69,71,74,228,229) have mixed types. Specify dtype option on import or set low_memory=False.
  for cdna_chunk in pd.read_csv(cdna_file, chunksize=cdna_chunksize):
<ipython-input-1-05b141c44fa0>:32: DtypeWarning: Columns (15,16,17,18,19,37,45,46,47,56,58,66,71,228) have mixed types. Specify dtype option on import or set low_memory=False.
  for cdna_chunk in pd.read_csv(cdna_file, chunksize=cdna_chunksize):
<ipython-input-1-05b141c44fa0>:32: DtypeWarning: Columns (7,12,13,15,16,17,18,19,29,37,45,46,47,56,58,71,228) have mixed types. Specify dtype option on import or set low_memory=False.
  for cdna_chunk in pd.read_csv(cdna_file, chunksize

In [ ]:
import pandas as pd

# Path to the merged dataset
merged_file = '/content/drive/My Drive/merged_data.csv'

# Load the first few rows to inspect column names
merged_sample = pd.read_csv(merged_file, nrows=5)
print("Column Names in Merged Dataset:")
print(merged_sample.columns.tolist())


Column Names in Merged Dataset:
['customer_code', 'offer_id', 'offer_subid', 'batch_id', 'product_category', 'product_sub_category', 'send_timestamp', 'open_timestamp', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v66', 'v67', 'v68', 'v69', 'v70', 'v71', 'v72', 'v73', 'v74', 'v75', 'v76', 'v77', 'v78', 'v79', 'v80', 'v81', 'v82', 'v83', 'v84', 'v85', 'v86', 'v87', 'v88', 'v89', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v108', 'v109', 'v110', 'v111', 'v112', 'v113', 'v114', 'v115', 'v116', 'v117', 'v118', '

In [13]:
import pandas as pd

# File paths
merged_file = '/content/drive/My Drive/merged_data.csv'  # Path to the large merged dataset
cleaned_file = '/content/drive/My Drive/cleaned_merged_data_new.csv'  # Path to save the cleaned dataset

columns_to_keep = [
    'customer_code', 'offer_id', 'offer_subid',
    'product_category', 'product_sub_category',
    'batch_id', 'send_timestamp', 'open_timestamp', 'batch_date','v6','acc_date'
]

# Prepare the output file (write header initially)
header_written = False

# Process the dataset in chunks
chunksize = 1_500_000  # Adjust the chunk size based on your system's memory
for chunk in pd.read_csv(merged_file, chunksize=chunksize):
    print("Processing a new chunk...")

    # Rename v11 to acc_date
    chunk.rename(columns={'v11': 'acc_date'}, inplace=True)

    # Keep only the necessary columns.
    chunk = chunk[columns_to_keep]

    # Save the cleaned chunk to a new file
    chunk.to_csv(cleaned_file, mode='a', index=False, header=not header_written)
    header_written = True  # Write header only for the first chunk

print(f"Cleaned dataset saved in chunks to: {cleaned_file}")


<ipython-input-13-0cdb96796821>:19: DtypeWarning: Columns (14,19,20,22,23,24,25,26,36,44,52,53,54,63,65,73,76,78,81,235) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(merged_file, chunksize=chunksize):


Processing a new chunk...


<ipython-input-13-0cdb96796821>:19: DtypeWarning: Columns (22,23,24,25,26,36,44,52,53,54,63,65,73,78,235) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(merged_file, chunksize=chunksize):


Processing a new chunk...


<ipython-input-13-0cdb96796821>:19: DtypeWarning: Columns (14,19,20,22,23,24,25,26,36,44,52,53,54,63,65,73,76,78,81,235) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(merged_file, chunksize=chunksize):


...

Processing a new chunk...


<ipython-input-13-0cdb96796821>:19: DtypeWarning: Columns (14,19,20,22,23,24,25,26,44,52,53,54,63,65,73,78,235) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(merged_file, chunksize=chunksize):


Processing a new chunk...


<ipython-input-13-0cdb96796821>:19: DtypeWarning: Columns (14,19,20,22,23,24,25,26,44,52,53,54,63,65,73,78,235) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(merged_file, chunksize=chunksize):


Processing a new chunk...
Cleaned dataset saved in chunks to: /content/drive/My Drive/cleaned_merged_data_new.csv


In [14]:
import pandas as pd

# Path to the cleaned dataset
cleaned_file = '/content/drive/My Drive/cleaned_merged_data_new.csv'

# Load a small sample to inspect column names
sample_data = pd.read_csv(cleaned_file, nrows=5)

# Print the column names
print("Column Names in the Cleaned Dataset:")
print(sample_data.columns.tolist())

# Display the first few rows to inspect the data
print("Sample Data:")
print(sample_data.head())


Column Names in the Cleaned Dataset:
['customer_code', 'offer_id', 'offer_subid', 'product_category', 'product_sub_category', 'batch_id', 'send_timestamp', 'open_timestamp', 'batch_date', 'v6', 'acc_date']
Sample Data:
                                       customer_code      offer_id  \
0  d3353919b286e084c3a0ca42fa6aab9343e8d189ef0d7b...  AC_100003681   
1  82682e9d1af585f25b07d9eecbe956bb2ac944a1c4d598...  AC_100003681   
2  6567bba1596f04c63ddd01e6bf0c4b161efc39f1e043df...  AC_100003681   
3  3bbaae5f12bb27efdc5a54158c2df4970a949df28b932a...  AC_100003681   
4  70a48789ecc274b17fdd1c770c74905f456914285c40e9...  AC_100003681   

    offer_subid product_category product_sub_category  batch_id  \
0  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
1  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
2  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
3  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
4  VR_200008007        PL

In [17]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
# Path to the dataset
dataset_file = '/content/drive/My Drive/cleaned_merged_data_new.csv'

# Load the dataset
data = pd.read_csv(dataset_file)

# Handle string columns
string_columns = ['customer_code', 'offer_id', 'offer_subid', 'product_category', 'product_sub_category','v6']
for col in string_columns:
    data[col] = data[col].astype(str)

# Handle numeric columns
data['batch_id'] = pd.to_numeric(data['batch_id'], errors='coerce')

# Handle datetime columns
datetime_columns = ['send_timestamp', 'open_timestamp', 'batch_date','acc_date']
for col in datetime_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')

# Save the updated dataset
final_dataset_file = '/content/drive/My Drive/final_cleaned_processed_data_newedit.csv'
data.to_csv(final_dataset_file, index=False)

print(f"Final processed dataset saved to: {final_dataset_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Final processed dataset saved to: /content/drive/My Drive/final_cleaned_processed_data_newedit.csv


In [19]:
import pandas as pd
import numpy as np

# Path to the dataset
dataset_file = '/content/drive/My Drive/final_cleaned_processed_data_newedit.csv'

# Load the dataset
data = pd.read_csv(dataset_file)

# Step 1: Convert timestamps to datetime
data['send_timestamp'] = pd.to_datetime(data['send_timestamp'], errors='coerce')
data['open_timestamp'] = pd.to_datetime(data['open_timestamp'], errors='coerce')

# Step 2: Generate weekly time slots based on 3-hour intervals
def get_weekly_slot(timestamp):
    if pd.isna(timestamp):
        return "No Engagement"
    day_of_week = timestamp.dayofweek  # 0=Monday, 6=Sunday
    hour = timestamp.hour

    # Map 3-hour intervals to slots
    if 9 <= hour < 12:
        slot = 1
    elif 12 <= hour < 15:
        slot = 2
    elif 15 <= hour < 18:
        slot = 3
    elif 18 <= hour < 21:
        slot = 4
    else:
        return "No Engagement"

    # Calculate the slot number for the week
    return day_of_week * 4 + slot  # 1-28 slots (Monday slot_1 = 1, Sunday slot_28 = 28)

# Apply to send and open timestamps
data['send_time_slot'] = data['send_timestamp'].apply(get_weekly_slot)
data['open_time_slot'] = data['open_timestamp'].apply(get_weekly_slot)

# Step 3: Replace "No Engagement" with NaN and convert to numeric
data['send_time_slot'] = data['send_time_slot'].replace("No Engagement", np.nan)
data['open_time_slot'] = data['open_time_slot'].replace("No Engagement", np.nan)

# Convert time slots to numeric
data['send_time_slot'] = pd.to_numeric(data['send_time_slot'], errors='coerce')
data['open_time_slot'] = pd.to_numeric(data['open_time_slot'], errors='coerce')

# Step 4: Create the target variable (Same-Slot Engagement)
data['same_slot_engagement'] = (data['send_time_slot'] == data['open_time_slot']).astype(int)

# Step 5: Aggregate historical data for independent variables
# Count the number of times a customer engaged in each slot
historical_features = data.groupby(['customer_code', 'open_time_slot']).size().unstack(fill_value=0)
historical_features.columns = [f'slot_{int(col)}_engagements' for col in historical_features.columns]

# Merge historical features back into the dataset
data = data.merge(historical_features, how='left', left_on='customer_code', right_index=True)

# Step 6: Save the dataset with the target variable and features
final_dataset_with_target = '/content/drive/My Drive/dataset_with_same_slot_target_newedit.csv'
data.to_csv(final_dataset_with_target, index=False)

print(f"Dataset with same-slot target and historical features saved to: {final_dataset_with_target}")


<ipython-input-19-dd23afdd08a8>:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['send_time_slot'] = data['send_time_slot'].replace("No Engagement", np.nan)
<ipython-input-19-dd23afdd08a8>:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['open_time_slot'] = data['open_time_slot'].replace("No Engagement", np.nan)


Dataset with same-slot target and historical features saved to: /content/drive/My Drive/dataset_with_same_slot_target_newedit.csv


In [20]:
print(data.head())
print(data['same_slot_engagement'].value_counts())


                                       customer_code      offer_id  \
0  d3353919b286e084c3a0ca42fa6aab9343e8d189ef0d7b...  AC_100003681   
1  82682e9d1af585f25b07d9eecbe956bb2ac944a1c4d598...  AC_100003681   
2  6567bba1596f04c63ddd01e6bf0c4b161efc39f1e043df...  AC_100003681   
3  3bbaae5f12bb27efdc5a54158c2df4970a949df28b932a...  AC_100003681   
4  70a48789ecc274b17fdd1c770c74905f456914285c40e9...  AC_100003681   

    offer_subid product_category product_sub_category  batch_id  \
0  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
1  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
2  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
3  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   
4  VR_200008007        PL X-SELL        CROSS SELL PL  20000200   

       send_timestamp open_timestamp  batch_date         v6  ...  \
0 2024-06-01 09:32:36            NaT  2024-05-31  BENGALURU  ...   
1 2024-06-01 09:32:42            NaT    

In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from sklearn.metrics import average_precision_score
import random
# Mount Google Drive to access datasets
from google.colab import drive
drive.mount('/content/drive')
# File path
final_dataset_file = '/content/drive/My Drive/dataset_with_same_slot_target_newedit.csv'

# Define chunk size
chunk_size = 500_000
map_scores = []

# LightGBM parameters with strong regularization
params = {
    'objective': 'lambdarank',
    'metric': 'map',
    'boosting_type': 'gbdt',
    'device': 'cpu',
    'learning_rate': 0.03,
    'num_leaves': 10,
    'max_depth': 5,
    'lambda_l1': 2.0,
    'lambda_l2': 2.0,
    'min_child_samples': 150,
    'seed': 42
}

for chunk in pd.read_csv(final_dataset_file, chunksize=chunk_size, engine='python'):
    print(f"Processing chunk...")

    # Encode categorical features
    categorical_columns = ['offer_id', 'offer_subid', 'product_category', 'product_sub_category','v6']
    for col in categorical_columns:
        if col in chunk.columns:
            chunk[col] = chunk[col].astype('category').cat.codes

    # Encode batch_date
    if 'batch_date' in chunk.columns:
        chunk['batch_date'] = pd.to_datetime(chunk['batch_date'], errors='coerce')
        chunk['batch_date'] = chunk['batch_date'].astype('category').cat.codes
    if 'acc_date' in chunk.columns:
        chunk['acc_date'] = pd.to_datetime(chunk['acc_date'], errors='coerce')
        chunk['acc_date'] = chunk['acc_date'].astype('category').cat.codes
    # Create relevance target
    chunk['relevance'] = np.where(chunk['open_time_slot'] == chunk['send_time_slot'], 1, 0)

    # Check for potential data leakage
    correlation = X.corrwith(y)
    print("Feature Correlations with Target (Relevance):")
    print(correlation.sort_values(ascending=False))

    # Drop highly correlated features
    threshold = 0.9  # Stricter threshold for leakage
    leakage_candidates = correlation[abs(correlation) > threshold].index.tolist()
    if leakage_candidates:
        print("Dropping Leakage Features:", leakage_candidates)
        X = X.drop(columns=leakage_candidates, errors='ignore')

    print("Features before dropping in training:", X.columns.tolist())

    # Prepare features and target
    X = chunk.drop(columns=['relevance', 'send_timestamp', 'open_timestamp', 'customer_code',], errors='ignore')
    y = chunk['relevance']
    if 'v6' in chunk.columns:
        X['v6'] = chunk['v6']
    # Check for potential data leakage
    correlation = X.corrwith(y)
    print("Feature Correlations with Target (Relevance):")
    print(correlation.sort_values(ascending=False))
    if 'v6' in chunk.columns:
      city_relevance = chunk.groupby('v6')['relevance'].mean()
    print("Mean relevance by city (v6):")
    print(city_relevance)
    # Drop highly correlated features (from observation same_slot_engagement was making the model cheat the predicitons it was dropped accordingly in every iteration)
    threshold = 0.9  # Stricter threshold for leakage
    leakage_candidates = correlation[abs(correlation) > threshold].index.tolist()
    if leakage_candidates:
        print("Dropping Leakage Features:", leakage_candidates)
        X = X.drop(columns=leakage_candidates, errors='ignore')

    # Add noise to the target for debugging
    y = y.apply(lambda x: random.choice([0, 1]) if random.random() < 0.05 else x)

    # GroupKFold ensures no group overlaps between training and validation
    groups = chunk['customer_code'].astype('category').cat.codes
    gkf = GroupKFold(n_splits=10)

    for fold, (train_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
        print(f"Training fold {fold + 1}...")

        # Train and validation splits
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Recalculate group sizes for train and validation sets
        train_group = X_train.groupby(groups.iloc[train_idx]).size().tolist()
        val_group = X_val.groupby(groups.iloc[val_idx]).size().tolist()

        # LightGBM datasets
        train_data = lgb.Dataset(X_train, label=y_train, group=train_group)
        val_data = lgb.Dataset(X_val, label=y_val, group=val_group)

        # Train the model
        model = lgb.train(
            params,
            train_data,
            valid_sets=[train_data, val_data],
            num_boost_round=1000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=50),
                lgb.log_evaluation(period=50)
            ]
        )

        # Predict and calculate MAP
        y_pred = model.predict(X_val)
        map_score = average_precision_score(y_val, y_pred)
        map_scores.append(map_score)
        print(f"Fold {fold + 1} MAP: {map_score}")

average_map = np.mean(map_scores)
print(f"Average MAP: {average_map}")


Streaming output truncated to the last 5000 lines.
slot_1_engagements      0.188721
slot_9_engagements      0.172711
slot_24_engagements     0.113954
slot_16_engagements     0.113300
slot_20_engagements     0.107183
slot_28_engagements     0.106598
slot_12_engagements     0.105573
slot_4_engagements      0.102750
slot_8_engagements      0.097293
batch_id                0.015809
product_sub_category    0.015773
offer_id                0.010053
acc_date                0.004983
offer_subid             0.000796
product_category       -0.000213
v6                     -0.001036
batch_date             -0.007990
send_time_slot         -0.010786
open_time_slot         -0.023343
dtype: float64
Mean relevance by city (v6):
v6
-1       0.107444
 0       0.108108
 1       0.250000
 2       0.083333
 3       0.000000
           ...   
 1696    0.112360
 1697    0.000000
 1698    0.000000
 1699    0.000000
 1700    1.000000
Name: relevance, Length: 1702, dtype: float64
Dropping Leakage Features: ['sa

In [24]:
import os

# Define the file path to save the model in Google Drive
model_path = '/content/drive/My Drive/final_lightgbm_model_newedit.txt'

# Save the LightGBM model for future use
model.save_model(model_path)
print(f"Model saved to: {model_path}")


Model saved to: /content/drive/My Drive/final_lightgbm_model_newedit.txt


NOW LETS MOVE ONTO TO TEST DATA PREPARATION

In [8]:
import pandas as pd

# File paths in Google Drive
communication_file = '/content/drive/My Drive/IIT_ROUND2/Test_r2/test_action_history.csv'
cdna_file = '/content/drive/My Drive/IIT_ROUND2/Test_r2/test_cdna_data.csv'

# Load Communication file
print("Loading Communication file...")
comm_data = pd.read_csv(communication_file, engine='python')

# Decapitalize column names in Communication file
comm_data.columns = [col.lower() for col in comm_data.columns]
print("Decapitalized Communication File Columns:")
print(comm_data.columns.tolist())

# Load CDNA file
print("Loading CDNA file...")
cdna_data = pd.read_csv(cdna_file, engine='python')

# Decapitalize column names in CDNA file
cdna_data.columns = [col.lower() for col in cdna_data.columns]
print("Decapitalized CDNA File Columns:")
print(cdna_data.columns.tolist())

# Retain only necessary columns from CDNA
cdna_data = cdna_data[['customer_code', 'batch_date', 'v11','v6']]
print("Retained columns from CDNA file:", cdna_data.columns.tolist())

# Convert timestamps to datetime
print("Converting timestamps to datetime format...")
comm_data['send_timestamp'] = pd.to_datetime(comm_data['send_timestamp'], errors='coerce').dt.tz_localize(None)
cdna_data['batch_date'] = pd.to_datetime(cdna_data['batch_date'], errors='coerce').dt.tz_localize(None)

# Sort both datasets for asof merge
comm_data = comm_data.sort_values('send_timestamp')
cdna_data = cdna_data.sort_values('batch_date')

# Perform asof merge
print("Merging Communication and CDNA files...")
merged_data = pd.merge_asof(
    comm_data,
    cdna_data,
    by='customer_code',
    left_on='send_timestamp',
    right_on='batch_date',
    direction='backward'
)

# Save merged data
merged_output_file = '/content/drive/My Drive/test_merged_data_newedit.csv'
merged_data.to_csv(merged_output_file, index=False)

print(f"Merged data saved to: {merged_output_file}")


Loading Communication file...
Decapitalized Communication File Columns:
['customer_code', 'offer_id', 'offer_subid', 'batch_id', 'product_category', 'product_sub_category', 'send_timestamp', 'open_timestamp']
Loading CDNA file...
Decapitalized CDNA File Columns:
['customer_code', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18', 'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v66', 'v67', 'v68', 'v69', 'v70', 'v71', 'v72', 'v73', 'v74', 'v75', 'v76', 'v77', 'v78', 'v79', 'v80', 'v81', 'v82', 'v83', 'v84', 'v85', 'v86', 'v87', 'v88', 'v89', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', '

In [9]:
import pandas as pd

# Path to the test dataset
test_file = '/content/drive/My Drive/test_merged_data_newedit.csv'
output_file = '/content/drive/My Drive/test_renamed_data_newedit.csv'

# Load the test dataset
print("Loading test data...")
test_data = pd.read_csv(test_file)

# Rename columns to keep consistency with the training data
print("Renaming columns...")
if 'v11' in test_data.columns:
    test_data.rename(columns={'v11': 'acc_date'}, inplace=True)

# Save the updated dataset
print("Saving the updated test dataset...")
test_data.to_csv(output_file, index=False)

print(f"Updated test dataset saved to: {output_file}")


Loading test data...


<ipython-input-9-e772a8458776>:9: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(test_file)


Renaming columns...
Saving the updated test dataset...
Updated test dataset saved to: /content/drive/My Drive/test_renamed_data_newedit.csv


In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the saved model
model_file = '/content/drive/My Drive/final_lightgbm_model_newedit.txt'

# Load the model
print("Loading the model from Google Drive...")
model = lgb.Booster(model_file=model_file)
print("Model successfully loaded!")

# Paths to the test data and submission file
test_file = '/content/drive/My Drive/test_renamed_data_newedit.csv'
submission_file = '/content/drive/My Drive/IIT_ROUND2/Test_r2/test_customers.csv'

# Function to generate weekly slots (same as in training)
def get_weekly_slot(timestamp):
    if pd.isna(timestamp):
        return "No Engagement"
    day_of_week = timestamp.dayofweek  # 0=Monday, 6=Sunday
    hour = timestamp.hour

    # Map 3-hour intervals to slots
    if 9 <= hour < 12:
        slot = 1
    elif 12 <= hour < 15:
        slot = 2
    elif 15 <= hour < 18:
        slot = 3
    elif 18 <= hour < 21:
        slot = 4
    else:
        return "No Engagement"

    # Calculate the slot number for the week
    return day_of_week * 4 + slot  # 1-28 slots (Monday slot_1 = 1, Sunday slot_28 = 28)

# Process the test data in chunks
chunk_size = 100000  # Adjust based on memory availability
predictions = []

print("Processing test data in chunks for predictions...")
for chunk in pd.read_csv(test_file, chunksize=chunk_size, engine='python'):
    print(f"Processing chunk with {len(chunk)} rows...")

    # Step 1: Convert necessary columns to strings
    string_columns = ['customer_code', 'offer_id', 'offer_subid', 'product_category', 'product_sub_category', 'v6']
    for col in string_columns:
        if col in chunk.columns:
            chunk[col] = chunk[col].astype(str)

    # Step 2: Convert timestamps to datetime
    chunk['send_timestamp'] = pd.to_datetime(chunk['send_timestamp'], errors='coerce')
    chunk['open_timestamp'] = pd.to_datetime(chunk['open_timestamp'], errors='coerce')

    # Step 3: Generate time slots using the provided logic
    chunk['send_time_slot'] = chunk['send_timestamp'].apply(get_weekly_slot)
    chunk['open_time_slot'] = chunk['open_timestamp'].apply(get_weekly_slot)

    # Step 4: Replace "No Engagement" with NaN and convert to numeric
    chunk['send_time_slot'] = chunk['send_time_slot'].replace("No Engagement", np.nan)
    chunk['open_time_slot'] = chunk['open_time_slot'].replace("No Engagement", np.nan)
    chunk['send_time_slot'] = pd.to_numeric(chunk['send_time_slot'], errors='coerce')
    chunk['open_time_slot'] = pd.to_numeric(chunk['open_time_slot'], errors='coerce')

    # Step 5: Encode numeric columns
    if 'batch_id' in chunk.columns:
        chunk['batch_id'] = pd.to_numeric(chunk['batch_id'], errors='coerce')

    # Step 6: Convert datetime columns and encode
    if 'batch_date' in chunk.columns:
        chunk['batch_date'] = pd.to_datetime(chunk['batch_date'], errors='coerce')
        chunk['batch_date'] = chunk['batch_date'].astype('category').cat.codes
    if 'acc_date' in chunk.columns:
        chunk['acc_date'] = pd.to_datetime(chunk['acc_date'], errors='coerce')
        chunk['acc_date'] = chunk['acc_date'].astype('category').cat.codes

    # Step 7: Encode categorical columns
    categorical_columns = ['offer_id', 'offer_subid', 'product_category', 'product_sub_category', 'v6']
    for col in categorical_columns:
        if col in chunk.columns:
            chunk[col] = chunk[col].astype('category').cat.codes

    # Step 8: Expand dataset for all 28 slots
    print("Expanding dataset for all 28 slots in this chunk...")
    all_slots = pd.DataFrame({
        'customer_code': np.repeat(chunk['customer_code'].unique(), 28),
        'slot': np.tile(range(1, 29), len(chunk['customer_code'].unique()))
    })

    # Merge expanded slots with the chunk
    expanded_chunk = all_slots.merge(chunk, on='customer_code', how='left')

    # Add slot-specific features
    for slot in range(1, 29):
        expanded_chunk[f'slot_{slot}_engagements'] = chunk.get(f'slot_{slot}_engagements', 0)

    # Ensure consistency with training features
    final_features = [
        'offer_id', 'offer_subid', 'product_category', 'product_sub_category', 'batch_id',
        'batch_date', 'send_time_slot', 'open_time_slot', 'acc_date', 'v6',
        'slot_1_engagements', 'slot_2_engagements', 'slot_3_engagements', 'slot_4_engagements',
        'slot_5_engagements', 'slot_6_engagements', 'slot_7_engagements', 'slot_8_engagements',
        'slot_9_engagements', 'slot_10_engagements', 'slot_11_engagements', 'slot_12_engagements',
        'slot_13_engagements', 'slot_14_engagements', 'slot_15_engagements', 'slot_16_engagements',
        'slot_17_engagements', 'slot_18_engagements', 'slot_19_engagements', 'slot_20_engagements',
        'slot_21_engagements', 'slot_22_engagements', 'slot_23_engagements', 'slot_24_engagements',
        'slot_25_engagements', 'slot_26_engagements', 'slot_27_engagements', 'slot_28_engagements'
    ]

    # Drop unnecessary columns and prepare features for prediction
    X_chunk = expanded_chunk[final_features].copy()

    # Predict engagement probabilities for the chunk
    print("Predicting engagement probabilities for this chunk...")
    expanded_chunk['engagement_probability'] = model.predict(X_chunk)

    # Append chunk predictions
    predictions.append(expanded_chunk[['customer_code', 'slot', 'engagement_probability']])

# Combine all predictions
print("Combining all chunk predictions...")
all_predictions = pd.concat(predictions, ignore_index=True)

# Rank slots by engagement probability
print("Ranking slots by engagement probability...")
ranked_predictions = all_predictions.groupby('customer_code').apply(
    lambda x: x.sort_values('engagement_probability', ascending=False)['slot'].unique().tolist()
).reset_index()

# Rename columns for submission
ranked_predictions.columns = ['customer_code', 'predicted_slots_order']
ranked_predictions['predicted_slots_order'] = ranked_predictions['predicted_slots_order'].apply(
    lambda slots: [f"slot_{slot}" for slot in slots]
)

# Load the submission file
print("Loading submission file...")
submission_data = pd.read_csv(submission_file)
submission_data.columns = [col.lower() for col in submission_data.columns]

# Append predictions to submission file
print("Appending predictions to submission file...")
submission_data = submission_data.merge(ranked_predictions, on='customer_code', how='left')

# Save the updated submission file
updated_submission_file = '/content/drive/My Drive/updated_submission_newedit4.csv'
submission_data.to_csv(updated_submission_file, index=False)
print(f"Updated submission file saved to: {updated_submission_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading the model from Google Drive...
Model successfully loaded!
Processing test data in chunks for predictions...
Processing chunk with 100000 rows...


<ipython-input-10-dae10067497a>:66: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk['send_time_slot'] = chunk['send_time_slot'].replace("No Engagement", np.nan)
<ipython-input-10-dae10067497a>:67: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk['open_time_slot'] = chunk['open_time_slot'].replace("No Engagement", np.nan)


Expanding dataset for all 28 slots in this chunk...
Predicting engagement probabilities for this chunk...
Processing chunk with 100000 rows...


<ipython-input-10-dae10067497a>:66: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk['send_time_slot'] = chunk['send_time_slot'].replace("No Engagement", np.nan)
<ipython-input-10-dae10067497a>:67: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk['open_time_slot'] = chunk['open_time_slot'].replace("No Engagement", np.nan)


Expanding dataset for all 28 slots in this chunk...
Predicting engagement probabilities for this chunk...
Processing chunk with 100000 rows...


...

Expanding dataset for all 28 slots in this chunk...
Predicting engagement probabilities for this chunk...
Processing chunk with 56503 rows...


<ipython-input-10-dae10067497a>:67: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk['open_time_slot'] = chunk['open_time_slot'].replace("No Engagement", np.nan)


Expanding dataset for all 28 slots in this chunk...
Predicting engagement probabilities for this chunk...
Combining all chunk predictions...
Ranking slots by engagement probability...


<ipython-input-10-dae10067497a>:132: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ranked_predictions = all_predictions.groupby('customer_code').apply(


Loading submission file...
Appending predictions to submission file...
Updated submission file saved to: /content/drive/My Drive/updated_submission_newedit4.csv


In [ ]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Path to the CSV file in Google Drive
csv_file_path = '/content/drive/My Drive/updated_submission_newedit4.csv'

# Read the CSV file
print("Loading CSV file...")
data = pd.read_csv(csv_file_path)

# Display the first few rows of the dataset
print("First few rows of the dataset:")
print(data.head())
